## Contents
- [Importing and Cleaning Land Area](#Importing-and-Cleaning-Land-Area)  
- [Importing and Cleaning DP05 Data](#Importing-and-Cleaning-DP05-Data)  
- [Create 'SEX AND AGE' Dataframe](#Create-'SEX-AND-AGE'-Dataframe)
- [Create 'RACE' Dataframe](#Create-'RACE'-Dataframe)
- [Importing and Cleaning DP03 Data](#Importing-and-Cleaning-DP03-Data) 
- [Create 'INCOME' Dataframe](#Create-'INCOME'-Dataframe)
- [Create 'HEALTH INSURANCE' Dataframe](#Create-'HEALTH-INSURANCE'-Dataframe)
- [Importing and Cleaning Obesity Data](#Importing-and-Cleaning-Obesity-Data)
- [Importing and Cleaning Testing Data](#Importing-and-Cleaning-Testing-Data) 

In [1]:
import pandas as pd
import numpy as np

# Importing and Cleaning Land Area

I contacted the Census Burearu on slack for land data ('AREALAND' parameter wasn't working for the Decennial 2010), and they referred me to Gazetteer for land data.
- [Gazetteer 2018 Geographic Data](https://www.census.gov/geographies/reference-files/time-series/geo/gazetteer-files.2018.html)

In [2]:
# Read in land data as dataframes.
tx_land = pd.read_csv('../data/preprocessing/raw_area_tx.csv')
ca_land = pd.read_csv('../data/preprocessing/raw_area_ca.csv')
fl_land = pd.read_csv('../data/preprocessing/raw_area_fl.csv')
il_land = pd.read_csv('../data/preprocessing/raw_area_il.csv')
ny_land = pd.read_csv('../data/preprocessing/raw_area_ny.csv')

# 'ALAND' is in square meters. 
# To convert square meters to square miles, divide by 2_589_988.
tx_land.head(2)

USPS  GEOID  ANSICODE             NAME       ALAND    AWATER  ALAND_SQMI  \
0   TX  48001   1383786  Anderson County  2752198848  39810264    1062.630   
1   TX  48003   1383787   Andrews County  3886850264    957039    1500.721   

   AWATER_SQMI  
0       15.371  
1        0.370

In [3]:
ca_land.head(2)

USPS  GEOID  ANSICODE            NAME       ALAND     AWATER  ALAND_SQMI  \
0   CA   6001   1675839  Alameda County  1909598013  216923745      737.30   
1   CA   6003   1675840   Alpine County  1912292630   12557304      738.34   

   AWATER_SQMI  
0       83.755  
1        4.848

In [4]:
fl_land.head(2)

USPS  GEOID  ANSICODE            NAME       ALAND     AWATER  ALAND_SQMI  \
0   FL  12001    308548  Alachua County  2266324954  242877007     875.033   
1   FL  12003    306920    Baker County  1515738968    9686120     585.230   

   AWATER_SQMI  
0       93.775  
1        3.740

In [5]:
il_land.head(2)

USPS  GEOID  ANSICODE              NAME       ALAND    AWATER  ALAND_SQMI  \
0   IL  17001    424202      Adams County  2214804824  41767689     855.141   
1   IL  17003    424203  Alexander County   609996948  44237171     235.521   

   AWATER_SQMI  
0       16.127  
1       17.080

In [6]:
ny_land.head(2)

USPS  GEOID  ANSICODE             NAME       ALAND    AWATER  ALAND_SQMI  \
0   NY  36001    974099    Albany County  1354120790  27124553     522.829   
1   NY  36003    974100  Allegany County  2666029673  12999275    1029.360   

   AWATER_SQMI  
0       10.473  
1        5.019

In [7]:
land = pd.concat([tx_land, ca_land, fl_land, il_land, ny_land])

In [8]:
land.head(2)

USPS  GEOID  ANSICODE             NAME       ALAND    AWATER  ALAND_SQMI  \
0   TX  48001   1383786  Anderson County  2752198848  39810264    1062.630   
1   TX  48003   1383787   Andrews County  3886850264    957039    1500.721   

   AWATER_SQMI  
0       15.371  
1        0.370

In [9]:
land['USPS'].value_counts()

TX    254
IL    102
FL     67
NY     62
CA     58
Name: USPS, dtype: int64

In [10]:
land['state'] = land['USPS'].str.replace('TX', ', Texas').str.replace('FL', ', Florida').str.replace('IL', ', Illinois').str.replace('NY', ', New York').str.replace('CA', ', California')

In [11]:
land.head(2)

USPS  GEOID  ANSICODE             NAME       ALAND    AWATER  ALAND_SQMI  \
0   TX  48001   1383786  Anderson County  2752198848  39810264    1062.630   
1   TX  48003   1383787   Andrews County  3886850264    957039    1500.721   

   AWATER_SQMI    state  
0       15.371  , Texas  
1        0.370  , Texas

In [12]:
land['county_state'] = land['NAME'] + land['state']

In [13]:
land.head(2)

USPS  GEOID  ANSICODE             NAME       ALAND    AWATER  ALAND_SQMI  \
0   TX  48001   1383786  Anderson County  2752198848  39810264    1062.630   
1   TX  48003   1383787   Andrews County  3886850264    957039    1500.721   

   AWATER_SQMI    state            county_state  
0       15.371  , Texas  Anderson County, Texas  
1        0.370  , Texas   Andrews County, Texas

In [14]:
# Drop all the columns except for county name and land square miles.
land = land[['county_state', 'ALAND_SQMI']]
land.head(2)

county_state  ALAND_SQMI
0  Anderson County, Texas    1062.630
1   Andrews County, Texas    1500.721

In [15]:
# Rename the columns.
land = land.rename(columns={'county_state': 'Geographic Area Name', 'ALAND_SQMI': 'sq_mi'})

In [16]:
land

Geographic Area Name     sq_mi
0         Anderson County, Texas  1062.630
1          Andrews County, Texas  1500.721
2         Angelina County, Texas   797.782
3          Aransas County, Texas   252.049
4           Archer County, Texas   903.289
..                           ...       ...
57   Washington County, New York   831.167
58        Wayne County, New York   603.829
59  Westchester County, New York   430.568
60      Wyoming County, New York   592.750
61        Yates County, New York   338.143

[543 rows x 2 columns]

In [17]:
# Set the county to the index
land = land.set_index('Geographic Area Name')

In [18]:
land.to_csv('../data/preprocessing/cleaned_area_five_states.csv')

# Importing and Cleaning DP05 Data

In [40]:
# Read in data with heads as dataframe, setting the geo name to the idex.
dp05 = pd.read_csv('../data/preprocessing/raw_dp05_with_headers_five_states.csv', index_col=0)
dp05.head(2)

Percent Margin of Error!!SEX AND AGE!!Total population!!65 years and over!!Female  \
Geographic Area Name                                                                                      
Austin County, Texas                                                0.6                                   
Kenedy County, Texas                                               20.5                                   

                      Estimate!!SEX AND AGE!!Total population!!65 years and over!!Sex ratio (males per 100 females)  \
Geographic Area Name                                                                                                  
Austin County, Texas                                               87.0                                               
Kenedy County, Texas                                               34.7                                               

                      Margin of Error!!SEX AND AGE!!Total population!!65 years and over!!Sex ratio (males per 100 females)  \
Geographic Area Name                                                                                                         
Austin County, Texas                                                1.9                                                      
Kenedy County, Texas                                               38.9                                                      

                      Percent Estimate!!SEX AND AGE!!Total population!!65 years and over!!Sex ratio (males per 100 females)  \
Geographic Area Name                                                                                                          
Austin County, Texas                                         -888888888                                                       
Kenedy County, Texas                                         -888888888                                                       

                      Percent Margin of Error!!SEX AND AGE!!Total population!!65 years and over!!Sex ratio (males per 100 females)  \
Geographic Area Name                                                                                                                 
Austin County, Texas                                         -888888888                                                              
Kenedy County, Texas                                         -888888888                                                              

                      Estimate!!RACE!!Total population  \
Geographic Area Name                                     
Austin County, Texas                             29565   
Kenedy County, Texas                               595   

                      Margin of Error!!RACE!!Total population  \
Geographic Area Name                                            
Austin County, Texas                               -555555555   
Kenedy County, Texas                                      181   

                      Percent Estimate!!RACE!!Total population  \
Geographic Area Name                                             
Austin County, Texas                                     29565   
Kenedy County, Texas                                       595   

                      Percent Margin of Error!!RACE!!Total population  \
Geographic Area Name                                                    
Austin County, Texas                                       -888888888   
Kenedy County, Texas                                       -888888888   

                      Estimate!!RACE!!Total population!!One race  ...  \
Geographic Area Name                                              ...   
Austin County, Texas                                       29004  ...   
Kenedy County, Texas                                         595  ...   

                      DP05_0004PMA  DP05_0004PEA  DP05_0018PMA  DP05_0018PEA  \
Geographic Area Name                                                           
Austin County, Texas           (X)           (X)           (

In [41]:
dp05.info()

<class 'pandas.core.frame.DataFrame'>
Index: 542 entries, Austin County, Texas to Mason County, Illinois
Columns: 377 entries, Percent Margin of Error!!SEX AND AGE!!Total population!!65 years and over!!Female to county
dtypes: float64(162), int64(196), object(19)
memory usage: 1.6+ MB


In [42]:
# Replace !! in column titles with empty space.
dp05.columns = dp05.columns.str.replace('!!', ' ')

In [43]:
# Drop the county and state number columns.
dp05 = dp05.drop(columns= ['county', 'state'])

## Replace -888888888, -555555555, and (X) with NaN

-888888888, or '(X)' means that the estimate is not applicable or not available.  
-555555555, or '*****' entry in the margin of error column indicates that the estimate is controlled. A statistical test for sampling variability is not appropriate.  

In [44]:
# Replace -888888888, -555555555, and '(X)' with NaN.
dp05 = dp05.replace([-888888888, -555555555, '(X)'], np.nan)

## Drop Margin of Error Columns

In [45]:
# Drop columns that contain 'Margin' in the name.
# https://stackoverflow.com/questions/406230/regular-expression-to-match-a-line-that-doesnt-contain-a-word
dp05 = dp05.filter(regex='^((?!Margin).)*$', axis=1)

## Drop Percent Columns

In [46]:
# Drop columns that contain 'Percent' in the name.
# Any percentages will be recalculated after combining total counts.
dp05 = dp05.filter(regex='^((?!Percent).)*$', axis=1)

## Drop Columns with NaN Values

In [47]:
# Drop the columns that have NaN values.
dp05 = dp05.loc[:, (dp05.isna().sum() < 1)]

# Create 'SEX AND AGE' Dataframe

In [48]:
# Extract the columns that have SEX AND AGE in the title
# and save to a new dataframe.
dp05_sex_age = dp05.filter(regex = '(SEX AND AGE)', axis=1)
dp05_sex_age.head(3)

Estimate SEX AND AGE Total population 65 years and over Sex ratio (males per 100 females)  \
Geographic Area Name                                                                                              
Austin County, Texas                                               87.0                                           
Kenedy County, Texas                                               34.7                                           
Nueces County, Texas                                               79.3                                           

                      Estimate SEX AND AGE Total population  \
Geographic Area Name                                          
Austin County, Texas                                  29565   
Kenedy County, Texas                                    595   
Nueces County, Texas                                 360486   

                      Estimate SEX AND AGE Total population Male  \
Geographic Area Name                                               
Austin County, Texas                                       14684   
Kenedy County, Texas                                         286   
Nueces County, Texas                                      178020   

                      Estimate SEX AND AGE Total population Female  \
Geographic Area Name                                                 
Austin County, Texas                                         14881   
Kenedy County, Texas                                           309   
Nueces County, Texas                                        182466   

                      Estimate SEX AND AGE Total population Sex ratio (males per 100 females)  \
Geographic Area Name                                                                            
Austin County, Texas                                               98.7                         
Kenedy County, Texas                                               92.6                         
Nueces County, Texas                                               97.6                         

                      Estimate SEX AND AGE Total population Under 5 years  \
Geographic Area Name                                                        
Austin County, Texas                                               1780     
Kenedy County, Texas                                                 85     
Nueces County, Texas                                              24665     

                      Estimate SEX AND AGE Total population 5 to 9 years  \
Geographic Area Name                                                       
Austin County, Texas                                               1960    
Kenedy County, Texas                                                 37    
Nueces County, Texas                                              25055    

                      Estimate SEX AND AGE Total population 10 to 14 years  \
Geographic Area Name                                                         
Austin County, Texas                                               2118      
Kenedy County, Texas                                                 40      
Nueces County, Texas                                              24806      

                      Estimate SEX AND AGE Total population 15 to 19 years  \
Geographic Area Name                                                         
Austin County, Texas                                               1861      
Kenedy County, Texas                                                 10      
Nueces County, Texas                                              25524      

                      Estimate SEX AND AGE Total population 20 to 24 years  \
Geographic Area Name                                                         
Austin County, Texas                                               1712      
Kenedy County, Texas                                                 10      
Nueces County, Texas                                              26397      

            

In [49]:
# Display information about the SEX and AGE dataframe
dp05_sex_age.info()

<class 'pandas.core.frame.DataFrame'>
Index: 542 entries, Austin County, Texas to Mason County, Illinois
Data columns (total 32 columns):
 #   Column                                                                                     Non-Null Count  Dtype  
---  ------                                                                                     --------------  -----  
 0   Estimate SEX AND AGE Total population 65 years and over Sex ratio (males per 100 females)  542 non-null    float64
 1   Estimate SEX AND AGE Total population                                                      542 non-null    int64  
 2   Estimate SEX AND AGE Total population Male                                                 542 non-null    int64  
 3   Estimate SEX AND AGE Total population Female                                               542 non-null    int64  
 4   Estimate SEX AND AGE Total population Sex ratio (males per 100 females)                    542 non-null    float64
 5   Estimate SEX AND 

In [50]:
# These are the columns to be used in the analysis.
columns_sex_age = [
    'Estimate SEX AND AGE Total population',
    'Estimate SEX AND AGE Total population Male',
    'Estimate SEX AND AGE Total population Female',
    'Estimate SEX AND AGE Total population Under 5 years',
    'Estimate SEX AND AGE Total population 5 to 9 years',
    'Estimate SEX AND AGE Total population 10 to 14 years',
    'Estimate SEX AND AGE Total population 15 to 19 years',
    'Estimate SEX AND AGE Total population 20 to 24 years',
    'Estimate SEX AND AGE Total population 25 to 34 years',
    'Estimate SEX AND AGE Total population 35 to 44 years',
    'Estimate SEX AND AGE Total population 45 to 54 years',
    'Estimate SEX AND AGE Total population 55 to 59 years',
    'Estimate SEX AND AGE Total population 60 to 64 years',
    'Estimate SEX AND AGE Total population 65 to 74 years',
    'Estimate SEX AND AGE Total population 75 to 84 years',
    'Estimate SEX AND AGE Total population 85 years and over',
    'Estimate SEX AND AGE Total population Median age (years)',
]

In [51]:
# Extract the important columns for gender and age analysis.
dp05_sex_age = dp05_sex_age[columns_sex_age]

In [52]:
# Display the dataframe
dp05_sex_age.head(2)

Estimate SEX AND AGE Total population  \
Geographic Area Name                                          
Austin County, Texas                                  29565   
Kenedy County, Texas                                    595   

                      Estimate SEX AND AGE Total population Male  \
Geographic Area Name                                               
Austin County, Texas                                       14684   
Kenedy County, Texas                                         286   

                      Estimate SEX AND AGE Total population Female  \
Geographic Area Name                                                 
Austin County, Texas                                         14881   
Kenedy County, Texas                                           309   

                      Estimate SEX AND AGE Total population Under 5 years  \
Geographic Area Name                                                        
Austin County, Texas                                               1780     
Kenedy County, Texas                                                 85     

                      Estimate SEX AND AGE Total population 5 to 9 years  \
Geographic Area Name                                                       
Austin County, Texas                                               1960    
Kenedy County, Texas                                                 37    

                      Estimate SEX AND AGE Total population 10 to 14 years  \
Geographic Area Name                                                         
Austin County, Texas                                               2118      
Kenedy County, Texas                                                 40      

                      Estimate SEX AND AGE Total population 15 to 19 years  \
Geographic Area Name                                                         
Austin County, Texas                                               1861      
Kenedy County, Texas                                                 10      

                      Estimate SEX AND AGE Total population 20 to 24 years  \
Geographic Area Name                                                         
Austin County, Texas                                               1712      
Kenedy County, Texas                                                 10      

                      Estimate SEX AND AGE Total population 25 to 34 years  \
Geographic Area Name                                                         
Austin County, Texas                                               3339      
Kenedy County, Texas                                                 95      

                      Estimate SEX AND AGE Total population 35 to 44 years  \
Geographic Area Name                                                         
Austin County, Texas                                               3275      
Kenedy County, Texas                                                 47      

                      Estimate SEX AND AGE Total population 45 to 54 years  \
Geographic Area Name                                                         
Austin County, Texas                                               3821      
Kenedy County, Texas                                                 75      

                      Estimate SEX AND AGE Total population 55 to 59 years  \
Geographic Area Name                                                         
Austin County, Texas                                               2327      
Kenedy County, Texas                                                 51      

                      Estimate SEX AND AGE Total population 60 to 64 years  \
Geographic Area Name                                                         
Austin County, Texas                                               1978      
Kenedy County, Texas                                                  9      

                      Estimate SEX AND AGE Total population 65 to 74 years  \
Geographic Area Nam

In [53]:
# Clean up column names.
dp05_sex_age.columns = dp05_sex_age.columns.str.lower().str.replace('estimate ', '').str.replace('sex and age', 'sex_age')

In [54]:
# Clean up column names.
dp05_sex_age.columns = dp05_sex_age.columns.str.replace('total population', 'pop').str.replace(' years', '').str.replace(' ', '_')

In [55]:
# Display columns.
list(dp05_sex_age.columns)

['sex_age_pop',
 'sex_age_pop_male',
 'sex_age_pop_female',
 'sex_age_pop_under_5',
 'sex_age_pop_5_to_9',
 'sex_age_pop_10_to_14',
 'sex_age_pop_15_to_19',
 'sex_age_pop_20_to_24',
 'sex_age_pop_25_to_34',
 'sex_age_pop_35_to_44',
 'sex_age_pop_45_to_54',
 'sex_age_pop_55_to_59',
 'sex_age_pop_60_to_64',
 'sex_age_pop_65_to_74',
 'sex_age_pop_75_to_84',
 'sex_age_pop_85_and_over',
 'sex_age_pop_median_age_(years)']

In [56]:
# Rename the median age column for the function.
dp05_sex_age = dp05_sex_age.rename(columns={'sex_age_pop_median_age_(years)': 'sex_age_median_age_in_years'})

In [68]:
# Export the cleaned county data for sex and age by percent.
dp05_sex_age.to_csv('../data/preprocessing/cleaned_dp05_sex_age_five_states.csv')

# Create 'RACE' Dataframe

In [59]:
# Extract the columns that don't have SEX AND AGE in the title
# and save to a new dataframe.
dp05_race = dp05.filter(regex='(RACE)', axis=1)
dp05_race.head(2)

Estimate RACE Total population  \
Geographic Area Name                                   
Austin County, Texas                           29565   
Kenedy County, Texas                             595   

                      Estimate RACE Total population One race  \
Geographic Area Name                                            
Austin County, Texas                                    29004   
Kenedy County, Texas                                      595   

                      Estimate RACE Total population Two or more races  \
Geographic Area Name                                                     
Austin County, Texas                                               561   
Kenedy County, Texas                                                 0   

                      Estimate RACE Total population One race.1  \
Geographic Area Name                                              
Austin County, Texas                                      29004   
Kenedy County, Texas                                        595   

                      Estimate RACE Total population One race White  \
Geographic Area Name                                                  
Austin County, Texas                                          23810   
Kenedy County, Texas                                            573   

                      Estimate RACE Total population One race Black or African American  \
Geographic Area Name                                                                      
Austin County, Texas                                               2626                   
Kenedy County, Texas                                                  0                   

                      Estimate RACE Total population One race American Indian and Alaska Native  \
Geographic Area Name                                                                              
Austin County, Texas                                                 48                           
Kenedy County, Texas                                                  0                           

                      Estimate RACE Total population One race American Indian and Alaska Native Cherokee tribal grouping  \
Geographic Area Name                                                                                                       
Austin County, Texas                                                  0                                                    
Kenedy County, Texas                                                  0                                                    

                      Estimate RACE Total population One race American Indian and Alaska Native Chippewa tribal grouping  \
Geographic Area Name                                                                                                       
Austin County, Texas                                                  0                                                    
Kenedy County, Texas                                                  0                                                    

                      Estimate RACE Total population One race American Indian and Alaska Native Navajo tribal grouping  \
Geographic Area Name                                                                                                     
Austin County, Texas                                                  0                                                  
Kenedy County, Texas                                                  0                                                  

                      ...  \
Geographic Area Name  ...   
Austin County, Texas  ...   
Kenedy County, Texas  ...   

                      Estimate HISPANIC OR LATINO AND RACE Total population Not Hispanic or Latino  \
Geographic Area Name                                                                                 
Austin County, Texas                                              21746                              
Kenedy County, Te

In [60]:
# Display information about the dataframe.
dp05_race.info()

<class 'pandas.core.frame.DataFrame'>
Index: 542 entries, Austin County, Texas to Mason County, Illinois
Data columns (total 46 columns):
 #   Column                                                                                                                                                       Non-Null Count  Dtype
---  ------                                                                                                                                                       --------------  -----
 0   Estimate RACE Total population                                                                                                                               542 non-null    int64
 1   Estimate RACE Total population One race                                                                                                                      542 non-null    int64
 2   Estimate RACE Total population Two or more races                                                                         

In [61]:
# The race columns are broken down into two basic parts, one witout hispanic, and one with. 
# The hispanic section appears to be more accurate.

columns_non_hispanic = [
    'Estimate RACE Total population',
    'Estimate RACE Total population One race White',
    'Estimate RACE Total population One race Black or African American',
    'Estimate RACE Total population One race American Indian and Alaska Native',
    'Estimate RACE Total population One race Asian',
    'Estimate RACE Total population One race Some other race',
    'Estimate RACE Total population Two or more races.1'
]

# This grouping seems to be better
columns_hispanic = [
    'Estimate RACE Total population',
    'Estimate HISPANIC OR LATINO AND RACE Total population Hispanic or Latino (of any race)',
    'Estimate HISPANIC OR LATINO AND RACE Total population Not Hispanic or Latino White alone',
    'Estimate HISPANIC OR LATINO AND RACE Total population Not Hispanic or Latino Black or African American alone',
    'Estimate HISPANIC OR LATINO AND RACE Total population Not Hispanic or Latino American Indian and Alaska Native alone',
    'Estimate HISPANIC OR LATINO AND RACE Total population Not Hispanic or Latino Asian alone',
    'Estimate HISPANIC OR LATINO AND RACE Total population Not Hispanic or Latino Native Hawaiian and Other Pacific Islander alone',
    'Estimate HISPANIC OR LATINO AND RACE Total population Not Hispanic or Latino Some other race alone',
    'Estimate HISPANIC OR LATINO AND RACE Total population Not Hispanic or Latino Two or more races'
]

In [62]:
# Extract the important columns for analysis.
dp05_race = dp05_race[columns_hispanic]

In [63]:
# Display the columns of the dataframe.
dp05_race.columns

Index(['Estimate RACE Total population',
       'Estimate HISPANIC OR LATINO AND RACE Total population Hispanic or Latino (of any race)',
       'Estimate HISPANIC OR LATINO AND RACE Total population Not Hispanic or Latino White alone',
       'Estimate HISPANIC OR LATINO AND RACE Total population Not Hispanic or Latino Black or African American alone',
       'Estimate HISPANIC OR LATINO AND RACE Total population Not Hispanic or Latino American Indian and Alaska Native alone',
       'Estimate HISPANIC OR LATINO AND RACE Total population Not Hispanic or Latino Asian alone',
       'Estimate HISPANIC OR LATINO AND RACE Total population Not Hispanic or Latino Native Hawaiian and Other Pacific Islander alone',
       'Estimate HISPANIC OR LATINO AND RACE Total population Not Hispanic or Latino Some other race alone',
       'Estimate HISPANIC OR LATINO AND RACE Total population Not Hispanic or Latino Two or more races'],
      dtype='object')

In [64]:
# Clean up column names.
dp05_race.columns = dp05_race.columns.str.lower().str.replace('estimate hispanic or latino and race total population', 'race pop')

In [65]:
# Clean up column names.
dp05_race.columns = dp05_race.columns.str.replace('not hispanic or latino ', '').str.replace('estimate race total population', 'race pop')

In [66]:
# Clean up column names.
dp05_race.columns = dp05_race.columns.str.replace('\(', '').str.replace('\)', '').str.replace(' ', '_')

In [67]:
# Display the columns of the dataframe.
list(dp05_race.columns)

['race_pop',
 'race_pop_hispanic_or_latino_of_any_race',
 'race_pop_white_alone',
 'race_pop_black_or_african_american_alone',
 'race_pop_american_indian_and_alaska_native_alone',
 'race_pop_asian_alone',
 'race_pop_native_hawaiian_and_other_pacific_islander_alone',
 'race_pop_some_other_race_alone',
 'race_pop_two_or_more_races']

In [69]:
# Export the cleaned county data for sex and age by percent.
dp05_race.to_csv('../data/preprocessing/cleaned_dp05_race_five_states.csv')

# Importing and Cleaning DP03 Data

In [88]:
# Read in data with heads as dataframe, setting the geo name to the idex.
dp03 = pd.read_csv('../data/preprocessing/raw_dp03_with_headers_five_states.csv', index_col=0)
dp03.head(3)

GEO_ID  \
Geographic Area Name                   
Austin County, Texas  0500000US48015   
Kenedy County, Texas  0500000US48261   
Nueces County, Texas  0500000US48355   

                      Estimate!!EMPLOYMENT STATUS!!Population 16 years and over  \
Geographic Area Name                                                              
Austin County, Texas                                              23354           
Kenedy County, Texas                                                428           
Nueces County, Texas                                             280990           

                      Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over  \
Geographic Area Name                                                                     
Austin County, Texas                                                108                  
Kenedy County, Texas                                                122                  
Nueces County, Texas                                                413                  

                      Percent Estimate!!EMPLOYMENT STATUS!!Population 16 years and over  \
Geographic Area Name                                                                      
Austin County, Texas                                              23354                   
Kenedy County, Texas                                                428                   
Nueces County, Texas                                             280990                   

                      Percent Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over  \
Geographic Area Name                                                                             
Austin County, Texas                                         -888888888                          
Kenedy County, Texas                                         -888888888                          
Nueces County, Texas                                         -888888888                          

                      Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force  \
Geographic Area Name                                                                              
Austin County, Texas                                              14475                           
Kenedy County, Texas                                                220                           
Nueces County, Texas                                             177352                           

                      Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force  \
Geographic Area Name                                                                                     
Austin County, Texas                                                413                                  
Kenedy County, Texas                                                 83                                  
Nueces County, Texas                                               1636                                  

                      Percent Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force  \
Geographic Area Name                                                                                      
Austin County, Texas                                               62.0                                   
Kenedy County, Texas                                               51.4                                   
Nueces County, Texas                                               63.1                                   

                      Percent Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force  \
Geographic Area Name                                                                                             
Austin County, Texas                                                1.8                                          
Kenedy County, Texas                                               13.4                                       

In [89]:
dp03.info()

<class 'pandas.core.frame.DataFrame'>
Index: 543 entries, Austin County, Texas to Mason County, Illinois
Columns: 643 entries, GEO_ID to county
dtypes: float64(246), int64(304), object(93)
memory usage: 2.7+ MB


In [90]:
# Replace !! in column titles with empty space.
dp03.columns = dp03.columns.str.replace('!!', ' ')

In [91]:
# Drop the state and county number columns.
dp03 = dp03.drop(columns= ['state', 'county', 'Geographic Area Name.1', 'GEO_ID'])

## Replace -888888888, -555555555, AND (X) with NaN

-888888888, or '(X)' means that the estimate is not applicable or not available.  
-555555555, or '*****' entry in the margin of error column indicates that the estimate is controlled. A statistical test for sampling variability is not appropriate.  

In [92]:
# Replace -888888888, -555555555, and '(X)' with NaN
dp03 = dp03.replace([-888888888, -555555555, '(X)'], np.nan)

## Drop Margin of Error Columns

In [93]:
# Drop columns that contain 'Margin' in the name.
# https://stackoverflow.com/questions/406230/regular-expression-to-match-a-line-that-doesnt-contain-a-word
dp03 = dp03.filter(regex='^((?!Margin).)*$', axis=1)

## Drop Percent Columns

In [94]:
# Drop columns that contain 'Percent' in the name.
# Any percentages will be calculated after recombining total counts
dp03 = dp03.filter(regex='^((?!Percent).)*$', axis=1)

## Drop Columns with NaN Values

In [95]:
# Drop the columns that have NaN values.
dp03 = dp03.loc[:, (dp03.isna().sum() < 1)]

In [96]:
# Display the first few rows of the dataframe
dp03.head(3)

Estimate EMPLOYMENT STATUS Population 16 years and over  \
Geographic Area Name                                                            
Austin County, Texas                                              23354         
Kenedy County, Texas                                                428         
Nueces County, Texas                                             280990         

                      Estimate EMPLOYMENT STATUS Population 16 years and over In labor force  \
Geographic Area Name                                                                           
Austin County, Texas                                              14475                        
Kenedy County, Texas                                                220                        
Nueces County, Texas                                             177352                        

                      Estimate EMPLOYMENT STATUS Population 16 years and over In labor force Civilian labor force  \
Geographic Area Name                                                                                                
Austin County, Texas                                              14475                                             
Kenedy County, Texas                                                220                                             
Nueces County, Texas                                             175891                                             

                      Estimate EMPLOYMENT STATUS Population 16 years and over In labor force Civilian labor force Employed  \
Geographic Area Name                                                                                                         
Austin County, Texas                                              13801                                                      
Kenedy County, Texas                                                220                                                      
Nueces County, Texas                                             165496                                                      

                      Estimate EMPLOYMENT STATUS Population 16 years and over In labor force Civilian labor force Unemployed  \
Geographic Area Name                                                                                                           
Austin County, Texas                                                674                                                        
Kenedy County, Texas                                                  0                                                        
Nueces County, Texas                                              10395                                                        

                      Estimate EMPLOYMENT STATUS Population 16 years and over In labor force Armed Forces  \
Geographic Area Name                                                                                        
Austin County, Texas                                                  0                                     
Kenedy County, Texas                                                  0                                     
Nueces County, Texas                                               1461                                     

                      Estimate EMPLOYMENT STATUS Population 16 years and over Not in labor force  \
Geographic Area Name                                                                               
Austin County, Texas                                               8879                            
Kenedy County, Texas                                                208                            
Nueces County, Texas                                             103638                            

                      Estimate EMPLOYMENT STATUS Civilian labor force  \
Geographic Area Name                                                    
Austin County, Texas                                            14475   
Kenedy Coun

Economic categories covered in DP03:
1. Employment Status
1. Commuting to Work
1. Occupation
1. Industry
1. Class of Worker
1. Income and Benefits
1. Health Insurance Coverage

In [97]:
# Display DP03 columns as a list.
list(dp03.columns)

['Estimate EMPLOYMENT STATUS Population 16 years and over',
 'Estimate EMPLOYMENT STATUS Population 16 years and over In labor force',
 'Estimate EMPLOYMENT STATUS Population 16 years and over In labor force Civilian labor force',
 'Estimate EMPLOYMENT STATUS Population 16 years and over In labor force Civilian labor force Employed',
 'Estimate EMPLOYMENT STATUS Population 16 years and over In labor force Civilian labor force Unemployed',
 'Estimate EMPLOYMENT STATUS Population 16 years and over In labor force Armed Forces',
 'Estimate EMPLOYMENT STATUS Population 16 years and over Not in labor force',
 'Estimate EMPLOYMENT STATUS Civilian labor force',
 'Estimate EMPLOYMENT STATUS Females 16 years and over',
 'Estimate EMPLOYMENT STATUS Females 16 years and over In labor force',
 'Estimate EMPLOYMENT STATUS Females 16 years and over In labor force Civilian labor force',
 'Estimate EMPLOYMENT STATUS Females 16 years and over In labor force Civilian labor force Employed',
 'Estimate EMP

# Create 'INCOME' Dataframe

In [98]:
# Extract the columns that have INCOME AND BENEFITS in the title
# and save to a new dataframe.
dp03_income = dp03.filter(regex = '(INCOME AND BENEFITS)', axis=1)
dp03_income.head(3)

Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households  \
Geographic Area Name                                                                                       
Austin County, Texas                                              11041                                    
Kenedy County, Texas                                                209                                    
Nueces County, Texas                                             128926                                    

                      Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households Less than $10,000  \
Geographic Area Name                                                                                                         
Austin County, Texas                                                482                                                      
Kenedy County, Texas                                                 25                                                      
Nueces County, Texas                                               8919                                                      

                      Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households $10,000 to $14,999  \
Geographic Area Name                                                                                                          
Austin County, Texas                                                459                                                       
Kenedy County, Texas                                                  4                                                       
Nueces County, Texas                                               6943                                                       

                      Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households $15,000 to $24,999  \
Geographic Area Name                                                                                                          
Austin County, Texas                                               1255                                                       
Kenedy County, Texas                                                 49                                                       
Nueces County, Texas                                              13180                                                       

                      Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households $25,000 to $34,999  \
Geographic Area Name                                                                                                          
Austin County, Texas                                                927                                                       
Kenedy County, Texas                                                 13                                                       
Nueces County, Texas                                              12922                                                       

                      Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households $35,000 to $49,999  \
Geographic Area Name                                                                                                          
Austin County, Texas                                               1186                                                       
Kenedy County, Texas                                                 71                                                       
Nueces County, Texas                                              16876                                                       

                      Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households $50,000 to $74,999  \
Geographic Area Name                                                                                                          
Austin County, Texas                                

In [99]:
# Display information about the dataframe.
dp03_income.info()

<class 'pandas.core.frame.DataFrame'>
Index: 543 entries, Austin County, Texas to Mason County, Illinois
Data columns (total 44 columns):
 #   Column                                                                                                                                                              Non-Null Count  Dtype
---  ------                                                                                                                                                              --------------  -----
 0   Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households                                                                                  543 non-null    int64
 1   Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households Less than $10,000                                                                543 non-null    int64
 2   Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households $10,000 to

In [100]:
# These are the columns to be used in the analysis.
columns_income = [
    'Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households',
    'Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households Less than $10,000',
    'Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households $10,000 to $14,999',
    'Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households $15,000 to $24,999',
    'Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households $25,000 to $34,999',
    'Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households $35,000 to $49,999',
    'Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households $50,000 to $74,999',
    'Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households $75,000 to $99,999',
    'Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households $100,000 to $149,999',
    'Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households $150,000 to $199,999',
    'Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households $200,000 or more',
    'Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households Median household income (dollars)',
    'Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households Mean household income (dollars)',
    'Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Per capita income (dollars)',
    'Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Median earnings for workers (dollars)',
    'Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Median earnings for male full-time, year-round workers (dollars)',
    'Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Median earnings for female full-time, year-round workers (dollars)'
]

In [101]:
# Extract the important columns for income analysis.
dp03_income = dp03_income[columns_income]

In [103]:
# Display the first few rows of the dataframe.
dp03_income.head(2)

Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households  \
Geographic Area Name                                                                                       
Austin County, Texas                                              11041                                    
Kenedy County, Texas                                                209                                    

                      Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households Less than $10,000  \
Geographic Area Name                                                                                                         
Austin County, Texas                                                482                                                      
Kenedy County, Texas                                                 25                                                      

                      Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households $10,000 to $14,999  \
Geographic Area Name                                                                                                          
Austin County, Texas                                                459                                                       
Kenedy County, Texas                                                  4                                                       

                      Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households $15,000 to $24,999  \
Geographic Area Name                                                                                                          
Austin County, Texas                                               1255                                                       
Kenedy County, Texas                                                 49                                                       

                      Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households $25,000 to $34,999  \
Geographic Area Name                                                                                                          
Austin County, Texas                                                927                                                       
Kenedy County, Texas                                                 13                                                       

                      Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households $35,000 to $49,999  \
Geographic Area Name                                                                                                          
Austin County, Texas                                               1186                                                       
Kenedy County, Texas                                                 71                                                       

                      Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households $50,000 to $74,999  \
Geographic Area Name                                                                                                          
Austin County, Texas                                               1851                                                       
Kenedy County, Texas                                                 16                                                       

                      Estimate INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS) Total households $75,000 to $99,999  \
Geographic Area Name                                                                                                          
Austin County, Texas                                               1651                                                       
Kenedy County, Texas                                                 23                                                       

                      Estimate

In [104]:
# Clean up column names.
dp03_income.columns = dp03_income.columns.str.lower().str.replace('estimate ', '')

In [105]:
# Clean up column names.
dp03_income.columns = dp03_income.columns.str.replace('income', 'inc').str.replace(' and benefits \(in 2018 inflation-adjusted dollars\)', '')

In [106]:
# Clean up column names.
dp03_income.columns = dp03_income.columns.str.replace('total ', '').str.replace(', ', '_').str.replace('$', '')

In [107]:
# Clean up column names.
dp03_income.columns = dp03_income.columns.str.replace('households', 'hhlds').str.replace('household', 'hhld').str.replace(',', '_')

In [108]:
# Clean up column names.
dp03_income.columns = dp03_income.columns.str.replace('inc hhlds median', 'inc median').str.replace('inc hhlds mean', 'inc mean')

In [109]:
# Clean up column names.
dp03_income.columns = dp03_income.columns.str.replace('\(', '').str.replace('\)', '').str.replace('for ', '').str.replace(' ', '_')

In [110]:
# Clean up column names.
dp03_income.columns = dp03_income.columns.str.replace('median', 'med').str.replace('full-time_year-round', 'full_yr')

In [111]:
# Clean up column names.
dp03_income.columns = dp03_income.columns.str.replace('dollars', 'dol').str.replace('earnings', 'earn')

In [112]:
# Display column names.
list(dp03_income.columns)

['inc_hhlds',
 'inc_hhlds_less_than_10_000',
 'inc_hhlds_10_000_to_14_999',
 'inc_hhlds_15_000_to_24_999',
 'inc_hhlds_25_000_to_34_999',
 'inc_hhlds_35_000_to_49_999',
 'inc_hhlds_50_000_to_74_999',
 'inc_hhlds_75_000_to_99_999',
 'inc_hhlds_100_000_to_149_999',
 'inc_hhlds_150_000_to_199_999',
 'inc_hhlds_200_000_or_more',
 'inc_med_hhld_inc_dol',
 'inc_mean_hhld_inc_dol',
 'inc_per_capita_inc_dol',
 'inc_med_earn_workers_dol',
 'inc_med_earn_male_full_yr_workers_dol',
 'inc_med_earn_female_full_yr_workers_dol']

In [113]:
# Export the cleaned county data for sex and age by percent.
dp03_income.to_csv('../data/preprocessing/cleaned_dp03_income_five_states.csv')

# Create 'HEALTH INSURANCE' Dataframe

In [114]:
# Extract the columns that have HEALTH INSURANCE in the title
# and save to a new dataframe.
dp03_ins = dp03.filter(regex = '(HEALTH INSURANCE)', axis=1)
dp03_ins.head(3)

Estimate HEALTH INSURANCE COVERAGE Civilian noninstitutionalized population  \
Geographic Area Name                                                                                
Austin County, Texas                                              29298                             
Kenedy County, Texas                                                595                             
Nueces County, Texas                                             355767                             

                      Estimate HEALTH INSURANCE COVERAGE Civilian noninstitutionalized population With health insurance coverage  \
Geographic Area Name                                                                                                               
Austin County, Texas                                              25749                                                            
Kenedy County, Texas                                                467                                                            
Nueces County, Texas                                             295165                                                            

                      Estimate HEALTH INSURANCE COVERAGE Civilian noninstitutionalized population With health insurance coverage With private health insurance  \
Geographic Area Name                                                                                                                                             
Austin County, Texas                                              20393                                                                                          
Kenedy County, Texas                                                212                                                                                          
Nueces County, Texas                                             208747                                                                                          

                      Estimate HEALTH INSURANCE COVERAGE Civilian noninstitutionalized population With health insurance coverage With public coverage  \
Geographic Area Name                                                                                                                                    
Austin County, Texas                                               8863                                                                                 
Kenedy County, Texas                                                276                                                                                 
Nueces County, Texas                                             119691                                                                                 

                      Estimate HEALTH INSURANCE COVERAGE Civilian noninstitutionalized population No health insurance coverage  \
Geographic Area Name                                                                                                             
Austin County, Texas                                               3549                                                          
Kenedy County, Texas                                                128                                                          
Nueces County, Texas                                              60602                                                          

                      Estimate HEALTH INSURANCE COVERAGE Civilian noninstitutionalized population Civilian noninstitutionalized population under 19 years  \
Geographic Area Name                                                                                                                                        
Austin County, Texas                                               7434                                                                                     
Kenedy County, Texas                                                167                                                                                     

In [115]:
# Display information about the dataframe.
dp03_ins.info()

<class 'pandas.core.frame.DataFrame'>
Index: 543 entries, Austin County, Texas to Mason County, Illinois
Data columns (total 24 columns):
 #   Column                                                                                                                                                                                                                      Non-Null Count  Dtype
---  ------                                                                                                                                                                                                                      --------------  -----
 0   Estimate HEALTH INSURANCE COVERAGE Civilian noninstitutionalized population                                                                                                                                                 543 non-null    int64
 1   Estimate HEALTH INSURANCE COVERAGE Civilian noninstitutionalized population With health insurance coverage          

In [116]:
# These are the columns to be used in the analysis.
columns_ins = [
    'Estimate HEALTH INSURANCE COVERAGE Civilian noninstitutionalized population',
    'Estimate HEALTH INSURANCE COVERAGE Civilian noninstitutionalized population With health insurance coverage',
    'Estimate HEALTH INSURANCE COVERAGE Civilian noninstitutionalized population With health insurance coverage With private health insurance',
    'Estimate HEALTH INSURANCE COVERAGE Civilian noninstitutionalized population With health insurance coverage With public coverage',
    'Estimate HEALTH INSURANCE COVERAGE Civilian noninstitutionalized population No health insurance coverage'
]

## NOTE: 

1. 'Estimate HEALTH INSURANCE COVERAGE Civilian noninstitutionalized population', (DP03_0095E)
2. 'Estimate HEALTH INSURANCE COVERAGE Civilian noninstitutionalized population With health insurance coverage', (DP03_0096E)
3. 'Estimate HEALTH INSURANCE COVERAGE Civilian noninstitutionalized population With health insurance coverage With private health insurance', (DP03_0097E)
4. 'Estimate HEALTH INSURANCE COVERAGE Civilian noninstitutionalized population With health insurance coverage With public coverage', (DP03_0098E)
5. 'Estimate HEALTH INSURANCE COVERAGE Civilian noninstitutionalized population No health insurance coverage' (DP03_0099E) 

Values 2 and 5 add up to value 1, as expected. But values 3 and 4 do not add up to value 2! Public/private coverage. does not add up to those with coverage since some people have both public and private coverage.

In [117]:
# Extract the important columns for analysis.
dp03_ins = dp03_ins[columns_ins]

In [118]:
# Display the first few rows of the dataframe.
dp03_ins.head(3)

Estimate HEALTH INSURANCE COVERAGE Civilian noninstitutionalized population  \
Geographic Area Name                                                                                
Austin County, Texas                                              29298                             
Kenedy County, Texas                                                595                             
Nueces County, Texas                                             355767                             

                      Estimate HEALTH INSURANCE COVERAGE Civilian noninstitutionalized population With health insurance coverage  \
Geographic Area Name                                                                                                               
Austin County, Texas                                              25749                                                            
Kenedy County, Texas                                                467                                                            
Nueces County, Texas                                             295165                                                            

                      Estimate HEALTH INSURANCE COVERAGE Civilian noninstitutionalized population With health insurance coverage With private health insurance  \
Geographic Area Name                                                                                                                                             
Austin County, Texas                                              20393                                                                                          
Kenedy County, Texas                                                212                                                                                          
Nueces County, Texas                                             208747                                                                                          

                      Estimate HEALTH INSURANCE COVERAGE Civilian noninstitutionalized population With health insurance coverage With public coverage  \
Geographic Area Name                                                                                                                                    
Austin County, Texas                                               8863                                                                                 
Kenedy County, Texas                                                276                                                                                 
Nueces County, Texas                                             119691                                                                                 

                      Estimate HEALTH INSURANCE COVERAGE Civilian noninstitutionalized population No health insurance coverage  
Geographic Area Name                                                                                                            
Austin County, Texas                                               3549                                                         
Kenedy County, Texas                                                128                                                         
Nueces County, Texas                                              60602

In [119]:
# Clean up column names.
dp03_ins.columns = dp03_ins.columns.str.lower().str.replace('estimate ', '')

In [120]:
# Clean up column names.
dp03_ins.columns = dp03_ins.columns.str.replace(\
    'health insurance coverage civilian noninstitutionalized population', 'health_ins_noninst_pop')

In [121]:
dp03_ins.columns = dp03_ins.columns.str.replace('with health insurance coverage', 'cov_yes')

In [122]:
# Clean up column names.
dp03_ins.columns = dp03_ins.columns.str.replace('no health insurance coverage', 'cov_no')

In [123]:
# Clean up column names.
dp03_ins.columns = dp03_ins.columns.str.replace('cov_yes with private health insurance', 'private')

In [124]:
# Clean up column names.
dp03_ins.columns = dp03_ins.columns.str.replace('cov_yes with public coverage', 'public')

In [125]:
# Clean up column names.
dp03_ins.columns = dp03_ins.columns.str.replace(' ', '_')

In [126]:
# Display the columns as a list.
list(dp03_ins.columns)

['health_ins_noninst_pop',
 'health_ins_noninst_pop_cov_yes',
 'health_ins_noninst_pop_private',
 'health_ins_noninst_pop_public',
 'health_ins_noninst_pop_cov_no']

In [128]:
# Export the cleaned county data for sex and age by percent.
dp03_ins.to_csv('../data/preprocessing/cleaned_dp03_insurance_five_states.csv')

# Importing and Cleaning Obesity Data

## Resources
[Obesity Data](https://gis.cdc.gov/grasp/diabetes/DiabetesAtlas.html)  
 Total, Adults Aged 20+ Years, Age-Adjusted Percentage, Texas, 2017  
 US Diabetes Surveillance System  
 www.cdc.gov/diabetes/data  

In [129]:
# Read in data with heads as dataframe, setting the geo name to the idex.
tx = pd.read_csv('../data/preprocessing/raw_obesity_tx.csv')
ca = pd.read_csv('../data/preprocessing/raw_obesity_ca.csv')
fl = pd.read_csv('../data/preprocessing/raw_obesity_fl.csv')
il = pd.read_csv('../data/preprocessing/raw_obesity_il.csv')
ny = pd.read_csv('../data/preprocessing/raw_obesity_ny.csv')

In [130]:
tx.head(2)

County  State  CountyFIPS  Percentage  Lower Limit   Upper Limit
0  Anderson County  Texas       48001        37.3         28.1          47.5
1   Andrews County  Texas       48003        31.3         20.0          44.2

In [131]:
ca.head(2)

County       State  CountyFIPS  Percentage  Lower Limit  \
0  Alameda County  California        6001         6.7          5.5   
1   Alpine County  California        6003         4.6          1.7   

    Upper Limit  
0           7.8  
1          11.2

In [132]:
fl.head(2)

County    State  CountyFIPS  Percentage  Lower Limit   Upper Limit
0  Alachua County  Florida       12001         9.6          8.2          11.3
1    Baker County  Florida       12003        13.6         11.0          16.3

In [133]:
il.head(2)

County     State  CountyFIPS  Percentage  Lower Limit  \
0      Adams County  Illinois       17001        11.4          6.8   
1  Alexander County  Illinois       17003         3.6          1.4   

    Upper Limit  
0          18.2  
1           7.5

In [134]:
ny.head(2)

County     State  CountyFIPS  Percentage  Lower Limit  \
0    Albany County  New York       36001         8.8          7.5   
1  Allegany County  New York       36003         9.3          7.8   

    Upper Limit  
0          10.2  
1          11.2

## Combine the dataframes

In [136]:
obesity = pd.concat([tx, ca, fl, il, ny])

In [137]:
obesity

County     State  CountyFIPS  Percentage  Lower Limit  \
0      Anderson County     Texas       48001        37.3         28.1   
1       Andrews County     Texas       48003        31.3         20.0   
2      Angelina County     Texas       48005        39.6         35.6   
3       Aransas County     Texas       48007        37.7         26.6   
4        Archer County     Texas       48009        28.3         18.9   
..                 ...       ...         ...         ...          ...   
57   Washington County  New York       36115        11.7          9.9   
58        Wayne County  New York       36117        11.0          9.2   
59  Westchester County  New York       36119         6.7          5.8   
60      Wyoming County  New York       36121        10.0          8.4   
61        Yates County  New York       36123        11.3          9.5   

     Upper Limit  
0           47.5  
1           44.2  
2           43.6  
3           49.8  
4           39.4  
..           ...  
57          13.8  
58          12.8  
59           7.6  
60          12.1  
61          13.6  

[543 rows x 6 columns]

## Create a county column using the lowercase county-only formatting

In [140]:
obesity['Geographic Area Name'] = obesity['County'] + ', ' + obesity['State']

In [141]:
obesity

County     State  CountyFIPS  Percentage  Lower Limit  \
0      Anderson County     Texas       48001        37.3         28.1   
1       Andrews County     Texas       48003        31.3         20.0   
2      Angelina County     Texas       48005        39.6         35.6   
3       Aransas County     Texas       48007        37.7         26.6   
4        Archer County     Texas       48009        28.3         18.9   
..                 ...       ...         ...         ...          ...   
57   Washington County  New York       36115        11.7          9.9   
58        Wayne County  New York       36117        11.0          9.2   
59  Westchester County  New York       36119         6.7          5.8   
60      Wyoming County  New York       36121        10.0          8.4   
61        Yates County  New York       36123        11.3          9.5   

     Upper Limit          Geographic Area Name  
0           47.5        Anderson County, Texas  
1           44.2         Andrews County, Texas  
2           43.6        Angelina County, Texas  
3           49.8         Aransas County, Texas  
4           39.4          Archer County, Texas  
..           ...                           ...  
57          13.8   Washington County, New York  
58          12.8        Wayne County, New York  
59           7.6  Westchester County, New York  
60          12.1      Wyoming County, New York  
61          13.6        Yates County, New York  

[543 rows x 7 columns]

In [142]:
obesity = obesity.rename(columns={'Percentage': 'obes_percent'})

In [143]:
obesity

County     State  CountyFIPS  obes_percent  Lower Limit  \
0      Anderson County     Texas       48001          37.3         28.1   
1       Andrews County     Texas       48003          31.3         20.0   
2      Angelina County     Texas       48005          39.6         35.6   
3       Aransas County     Texas       48007          37.7         26.6   
4        Archer County     Texas       48009          28.3         18.9   
..                 ...       ...         ...           ...          ...   
57   Washington County  New York       36115          11.7          9.9   
58        Wayne County  New York       36117          11.0          9.2   
59  Westchester County  New York       36119           6.7          5.8   
60      Wyoming County  New York       36121          10.0          8.4   
61        Yates County  New York       36123          11.3          9.5   

     Upper Limit          Geographic Area Name  
0           47.5        Anderson County, Texas  
1           44.2         Andrews County, Texas  
2           43.6        Angelina County, Texas  
3           49.8         Aransas County, Texas  
4           39.4          Archer County, Texas  
..           ...                           ...  
57          13.8   Washington County, New York  
58          12.8        Wayne County, New York  
59           7.6  Westchester County, New York  
60          12.1      Wyoming County, New York  
61          13.6        Yates County, New York  

[543 rows x 7 columns]

In [144]:
obesity['obes_percent'] = obesity['obes_percent'] / 100

In [146]:
obesity = obesity[['Geographic Area Name', 'obes_percent']]

In [150]:
obesity = obesity.set_index('Geographic Area Name')

In [151]:
obesity

obes_percent
Geographic Area Name                      
Anderson County, Texas               0.373
Andrews County, Texas                0.313
Angelina County, Texas               0.396
Aransas County, Texas                0.377
Archer County, Texas                 0.283
...                                    ...
Washington County, New York          0.117
Wayne County, New York               0.110
Westchester County, New York         0.067
Wyoming County, New York             0.100
Yates County, New York               0.113

[543 rows x 1 columns]

In [152]:
obesity.to_csv('../data/preprocessing/cleaned_obesity_five_states.csv')

# Importing and Cleaning Testing Data

In [79]:
tests = pd.read_csv('../data/preprocessing/Texas_total_tests_oct-22.csv')

In [80]:
tests.head(3)

COVID-19 Cumulative Total Tests Performed in Texas by County Unnamed: 1  \
0                                             County               13-Oct   
1                                           Anderson               28,600   
2                                            Andrews                1,843   

  Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5 Unnamed: 6 Unnamed: 7  \
0     14-Oct     15-Oct     16-Oct     17-Oct     18-Oct     19-Oct   
1     28,641     28,844     29,027     29,075     29,317     29,350   
2      1,854      1,888      1,894      1,903      1,915      1,925   

  Unnamed: 8 Unnamed: 9 Unnamed: 10 Unnamed: 11  
0     20-Oct     21-Oct      22-Oct      23-Oct  
1     29,371     29,485      29,843      29,964  
2      1,930      1,943       2,092       2,112

In [81]:
# Rename the columns using the first row.
tests = tests.rename(columns=tests.iloc[0])
tests

County     13-Oct     14-Oct     15-Oct     16-Oct     17-Oct  \
0      County     13-Oct     14-Oct     15-Oct     16-Oct     17-Oct   
1    Anderson     28,600     28,641     28,844     29,027     29,075   
2     Andrews      1,843      1,854      1,888      1,894      1,903   
3    Angelina     18,736     18,815     18,974     19,091     19,564   
4     Aransas      3,135      3,239      3,340      3,354      3,414   
..        ...        ...        ...        ...        ...        ...   
252     Young      3,045      3,063      3,189      3,201      3,223   
253    Zapata      3,948      3,958      4,074      4,208      4,218   
254    Zavala      2,248      2,264      2,305      2,329      2,345   
255   Unknown    113,478    113,579    113,719    113,765    113,798   
256     Total  7,719,407  7,769,036  7,885,463  7,953,016  8,014,408   

        18-Oct     19-Oct     20-Oct     21-Oct     22-Oct     23-Oct  
0       18-Oct     19-Oct     20-Oct     21-Oct     22-Oct     23-Oct  
1       29,317     29,350     29,371     29,485     29,843     29,964  
2        1,915      1,925      1,930      1,943      2,092      2,112  
3       20,161     20,196     20,211     20,269     20,325     21,026  
4        3,433      3,476      3,501      3,534      3,575      3,619  
..         ...        ...        ...        ...        ...        ...  
252      3,233      3,259      3,262      3,273      3,298      3,347  
253      4,232      4,287      4,291      4,307      4,324      4,346  
254      2,358      2,370      2,375      2,377      2,390      2,494  
255    113,824    113,906    113,933    114,111    114,179    114,247  
256  8,078,858  8,146,209  8,180,077  8,245,551  8,338,444  8,447,606  

[257 rows x 12 columns]

In [82]:
# Drop the first and last two rows.
tests = tests.iloc[1:255, :]

In [83]:
tests

County  13-Oct  14-Oct  15-Oct  16-Oct  17-Oct  18-Oct  19-Oct  20-Oct  \
1    Anderson  28,600  28,641  28,844  29,027  29,075  29,317  29,350  29,371   
2     Andrews   1,843   1,854   1,888   1,894   1,903   1,915   1,925   1,930   
3    Angelina  18,736  18,815  18,974  19,091  19,564  20,161  20,196  20,211   
4     Aransas   3,135   3,239   3,340   3,354   3,414   3,433   3,476   3,501   
5      Archer     956     964     990   1,003   1,013   1,015   1,031   1,033   
..        ...     ...     ...     ...     ...     ...     ...     ...     ...   
250      Wood   6,095   6,166   6,242   6,289   6,342   6,393   6,447   6,483   
251    Yoakum   1,439   1,444   1,446   1,448   1,478   1,479   1,480   1,481   
252     Young   3,045   3,063   3,189   3,201   3,223   3,233   3,259   3,262   
253    Zapata   3,948   3,958   4,074   4,208   4,218   4,232   4,287   4,291   
254    Zavala   2,248   2,264   2,305   2,329   2,345   2,358   2,370   2,375   

     21-Oct  22-Oct  23-Oct  
1    29,485  29,843  29,964  
2     1,943   2,092   2,112  
3    20,269  20,325  21,026  
4     3,534   3,575   3,619  
5     1,045   1,062   1,073  
..      ...     ...     ...  
250   6,554   6,699   6,804  
251   1,486   1,495   1,525  
252   3,273   3,298   3,347  
253   4,307   4,324   4,346  
254   2,377   2,390   2,494  

[254 rows x 12 columns]

In [84]:
# The final date to use is '2020-10-21'
tests = tests[['County', '21-Oct']]

In [85]:
tests['County'] = tests['County'].str.lower()

/home/lyzka/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [86]:
tests = tests.rename(columns={'County': 'county_name', '21-Oct': 'total_tests'})

In [87]:
tests

county_name total_tests
1      anderson      29,485
2       andrews       1,943
3      angelina      20,269
4       aransas       3,534
5        archer       1,045
..          ...         ...
250        wood       6,554
251      yoakum       1,486
252       young       3,273
253      zapata       4,307
254      zavala       2,377

[254 rows x 2 columns]

In [88]:
tests.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 1 to 254
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   county_name  254 non-null    object
 1   total_tests  254 non-null    object
dtypes: object(2)
memory usage: 4.1+ KB


In [89]:
# Define a function to clean the entries.
def clean_nums(text):
    new_string = ''
    for ch in text:
        if ch != ',':
            new_string = new_string + ch

    return int(new_string)

In [90]:
tests['total_tests'] = tests['total_tests'].apply(clean_nums)

In [91]:
tests.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 1 to 254
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   county_name  254 non-null    object
 1   total_tests  254 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 4.1+ KB


In [92]:
tests

county_name  total_tests
1      anderson        29485
2       andrews         1943
3      angelina        20269
4       aransas         3534
5        archer         1045
..          ...          ...
250        wood         6554
251      yoakum         1486
252       young         3273
253      zapata         4307
254      zavala         2377

[254 rows x 2 columns]

In [93]:
# Export the cleaned data.
tests.to_csv('../data/preprocessing/cleaned_tx_tests.csv', index=False)